## A. 
Data exploration
Explore the three datasets. Use your judgment as a human being to describe
what makes you able to tell spam apart from ham. Also explore the differences between the two classes of ham (easy and hard ham). What makes them different? Write approximately one paragraph.

1. ##### Difference between Ham and Spam
    Spam contains a lot of clickbaity titles with links that looks more than suspicious. Very sensational language and the senders adress looks very auto-generated for example: sabrina@mx3.1premio.com.
    Ham on the other hand is more legit and the emails looks like they are sent from either real people or real companies with a newsletter and you get the choice to unsubscribe or not.

2. ##### Difference between Hard and Easy Ham
    Easy ham seems to be mostly conversations between real life people, not alot of code, mostly natural speaking language
    Hard Ham contains more code, and contains mostly of commercials which makes it hard to judge if it's actually wanted by the reciever or not.

## B.
Perform an appropriate train-test split on the each of the datasets. We will use
the training sets to train a classifier, and evaluate its performance against the
test sets.

In [22]:
import pandas as pd
import os

# Creating lists and dicts to parse the text files
easy_ham_dictionary = {}
hard_ham_dictionary = {}
spam_dictionary = {}
easy_ham_texts = []
hard_ham_texts = []
spam_texts = []
easy_ham_path = os.getcwd() + "/20021010_easy_ham/easy_ham/"
hard_ham_path = os.getcwd() + "/20021010_hard_ham/hard_ham/"
spam_path = os.getcwd() + "/20021010_spam/spam/"

# Parsing the text files into lists
for file_name in os.listdir(easy_ham_path):
    f = open(easy_ham_path+file_name, "rb")
    text = f.read()
    easy_ham_texts.append(text)
    f.close()

for file_name in os.listdir(hard_ham_path):
    f = open(hard_ham_path+file_name, "rb")
    text = f.read()
    hard_ham_texts.append(text)
    f.close()

for file_name in os.listdir(spam_path):
    f = open(spam_path+file_name, "rb")
    text = f.read()
    spam_texts.append(text)
    f.close()

# Creating dicts from the lists
easy_ham_dictionary["easy_ham_texts"] = easy_ham_texts
hard_ham_dictionary["hard_ham_texts"] = hard_ham_texts
spam_dictionary["spam_texts"] = spam_texts

# Creating dataframes from the dicts
easy_ham_df = pd.DataFrame.from_dict(easy_ham_dictionary)
hard_ham_df = pd.DataFrame.from_dict(hard_ham_dictionary)
spam_df = pd.DataFrame.from_dict(spam_dictionary)

print(easy_ham_df)
print(spam_df)
print(hard_ham_df)


                                         easy_ham_texts
0     b'From exmh-workers-admin@redhat.com  Thu Aug ...
1     b'From Steve_Burt@cursor-system.com  Thu Aug 2...
2     b'From timc@2ubh.com  Thu Aug 22 13:52:59 2002...
3     b'From irregulars-admin@tb.tf  Thu Aug 22 14:2...
4     b'From exmh-users-admin@redhat.com  Thu Aug 22...
...                                                 ...
2546  b"From rssfeeds@jmason.org  Thu Oct 10 12:32:3...
2547  b'From rssfeeds@jmason.org  Thu Oct 10 12:32:3...
2548  b'From rssfeeds@jmason.org  Thu Oct 10 12:32:3...
2549  b"From rssfeeds@jmason.org  Thu Oct 10 12:32:3...
2550  b'From rssfeeds@jmason.org  Thu Oct 10 12:32:3...

[2551 rows x 1 columns]
                                            spam_texts
0    b'mv 1 00001.bfc8d64d12b325ff385cca8d07b84288\...
1    b'From 12a1mailbot1@web.de  Thu Aug 22 13:17:2...
2    b'From ilug-admin@linux.ie  Thu Aug 22 13:27:3...
3    b'From sabrina@mx3.1premio.com  Thu Aug 22 14:...
4    b'From wsup@playful.com